In [1]:
import os
from pathlib import Path
import math
import colossalai
import torch
import torch.nn as nn
import torch.nn.functional as F
from colossalai.core import global_context as gpc
from colossalai.logging import get_dist_logger
from colossalai.nn import CosineAnnealingLR
from colossalai.nn.metric import Accuracy
from colossalai.trainer import Trainer, hooks
from colossalai.utils import MultiTimer, get_dataloader
from torchvision import transforms
from torchvision.datasets import MNIST
from tqdm import tqdm

In [2]:
# Demo exponentially increase learning rate from low to high
def lrs(batch):
    low = math.log2(1e-5)
    high = math.log2(10)
    return 2**(low+(high-low)*batch/1000/5)
print(lrs(1), lrs(100), lrs(1000))

1.0027669229965856e-05 1.3182567385564057e-05 0.00015848931924611128


In [9]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits


def run(optimizer_method, scheduler_method, learning_rate):
    
    logger = get_dist_logger()
    d_name = f'{optimizer_method}_{scheduler_method}_{learning_rate}'

    # build 
    model = LeNet5(n_classes=10)

    # build dataloaders
    train_dataset = MNIST(
        root=Path('./tmp/'),
        download=True,
        transform = transforms.Compose([transforms.Resize((32, 32)),
                                  transforms.ToTensor()])
    )

    test_dataset = MNIST(
        root=Path('./tmp/'),
        train=False,
        transform = transforms.Compose([transforms.Resize((32, 32)),
                                  transforms.ToTensor()])
    )

    train_dataloader = get_dataloader(dataset=train_dataset,
                                      shuffle=True,
                                      batch_size=gpc.config.BATCH_SIZE,
                                      num_workers=1,
                                      pin_memory=True,
                                      )

    test_dataloader = get_dataloader(dataset=test_dataset,
                                      add_sampler=False,
                                      batch_size=gpc.config.BATCH_SIZE,
                                      num_workers=1,
                                      pin_memory=True,
                                      )

    # build criterion
    criterion = torch.nn.CrossEntropyLoss()

    # optimizer
    if optimizer_method.lower()=='sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
    elif optimizer_method.lower()=='adamw':
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=5e-4)
    else:
        raise NotImplementedError
    
    # lr_scheduler
    if scheduler_method.lower()=='lambda':
        #exponentially increase learning rate from low to high
        def lrs(batch):
            low = math.log2(1e-5)
            high = math.log2(10)
            return 2**(low+(high-low)*batch/len(train_dataloader)/gpc.config.NUM_EPOCHS)
        lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lrs)
    elif scheduler_method.lower()=='multistep':
        lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[i*len(train_dataloader)/2 for i in range(gpc.config.NUM_EPOCHS*2)], gamma=1.5)
    elif scheduler_method.lower()=='onecycle':
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer, max_lr=learning_rate/10,
            base_momentum=2, max_momentum=2,
            steps_per_epoch=len(train_dataloader),
            epochs=gpc.config.NUM_EPOCHS
        )
    else:
        raise NotImplementedError
    
    engine, train_dataloader, test_dataloader, _ = colossalai.initialize(
        model, optimizer, criterion, train_dataloader, test_dataloader
    )
    
    # build a timer to measure time
    timer = MultiTimer()

    # create a trainer object
    trainer = Trainer(
        engine=engine,
        timer=timer,
        logger=logger
    )

    # define the hooks to attach to the trainer
    hook_list = [
        hooks.LossHook(),
        hooks.LRSchedulerHook(lr_scheduler=lr_scheduler, by_epoch=False),
        # hooks.AccuracyHook(accuracy_func=Accuracy()),
        hooks.LogMetricByEpochHook(logger),
        hooks.LogMemoryByEpochHook(logger),
        hooks.LogTimingByEpochHook(timer, logger),

        # you can uncomment these lines if you wish to use them
        hooks.TensorboardHook(log_dir=f'./tb_logs/{d_name}', ranks=[0]),
        # hooks.SaveCheckpointHook(checkpoint_dir='./ckpt')
    ]

    # start training
    trainer.fit(
        train_dataloader=train_dataloader,
        epochs=gpc.config.NUM_EPOCHS,
        test_dataloader=test_dataloader,
        test_interval=1,
        hooks=hook_list,
        display_progress=True
    )
    
    logger.log_to_file(f'./tb_logs/{d_name}')

In [10]:
# Propose several learning rates for real training.
learning_rates = [0.1]
# Choose one optimizer
optimizer_methods = ['sgd','adamw']
# Choose two learning rate scheduling method 
scheduler_methods = ['lambda','multistep']

# Single launch
config = {'BATCH_SIZE':128,'NUM_EPOCHS':10}
colossalai.launch(config=config,rank=0,world_size=1,host='127.0.0.1',port=1234)

# Loop
for learning_rate in learning_rates:
    for optimizer_method in optimizer_methods:
        for scheduler_method in scheduler_methods:
            print('>>>>>>', optimizer_method, scheduler_method, learning_rate)
            run(optimizer_method, scheduler_method, learning_rate)

>>>>>> sgd onecycle 0.1


[03/29/22 03:07:01] INFO     colossalai - colossalai - 2022-03-29 03:07:01,521 ]8;id=746834;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py\logging.py]8;;\:]8;id=287968;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py#109\109]8;;\
                             INFO:                                                           
                             ========== Your Config ========                                 
                             {'BATCH_SIZE': 128, 'NUM_EPOCHS': 10}                           
                             ================================                                
                                                                                             

                    INFO     colossalai - colossalai - 2022-03-29 03:07:01,526 ]8;id=648262;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py\logging.py]8;;\:]8;id=293625;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py#109\109]8;;\
                             INFO: cuDNN benchmark = True, deterministic =                   
                             False                                                           

                    WARNING  colossalai - colossalai - 2022-03-29 03:07:01,530 ]8;id=42284;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py\logging.py]8;;\:]8;id=738505;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py#105\105]8;;\
                             WARNING: Initializing an non ZeRO model with                    
                             optimizer class                                                 

                    WARNING  colossalai - colossalai - 2022-03-29 03:07:01,533 ]8;id=246596;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py\logging.py]8;;\:]8;id=12237;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py#109\109]8;;\
                             WARNING: No PyTorch DDP or gradient handler is                  
                             set up, please make sure you do not need to                     
                             all-reduce the gradients after a training step.                 

...

                    INFO     colossalai - colossalai - 2022-03-29 03:09:02,487 ]8;id=307219;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py\logging.py]8;;\:]8;id=229293;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py#109\109]8;;\
                             INFO: Using TensorboardHook for training,                       
                             priority = 10                                                   

                    INFO     colossalai - colossalai - 2022-03-29 03:09:02,491 ]8;id=858829;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py\logging.py]8;;\:]8;id=705451;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py#109\109]8;;\
                             INFO: Lower value means higher priority for                     
                             calling hook function                                           

                    INFO     colossalai - colossalai - 2022-03-29 03:09:02,495 ]8;id=434022;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py\logging.py]8;;\:]8;id=83080;file:///home/fiona/anaconda3/envs/pytorch/lib/python3.9/site-packages/colossalai/logging/logging.py#105\105]8;;\
                             INFO: Before-train: GPU: allocated 11.49 MB, max                
                             allocated 11.49 MB, cached: 76.0 MB, max cached:                
                             76.0 MB                                                         

[Epoch 0 / Train]:  98%|████████████████████████████████████████████████████████████▊ | 460/469 [00:10<00:00, 45.31it/s]


KeyboardInterrupt: 